# Transform county data on professions into county data on skill levels

1. Load data and set it up
2. Prepare dictionary and helper function
3. Transform data and save it

# 1. Load Data

In [70]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [71]:
#libraries
import re
import pandas as pd
import numpy as np
from scipy.stats import chisquare
import seaborn as sns
import matplotlib.pyplot as plt

**Set up File Paths**
1. Loading
2. Saving

**Loading**

In [72]:
# Set up paths (load files from Classification folder)
classification_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\0.1 Classification Data\Different Skills"

# Set up path to control variables 
path_control_df = r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data"

# Set up path to ifo (iPEHD) files
ifo_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\0.3 ifo Data"

# Load data
occupation_detailed_df = pd.read_stata(f"{classification_path}\occupation_detailed.dta")
control_df = pd.read_stata(f"{path_control_df}\control.dta")

**Saving**

In [73]:
saving_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\2.2 Saving the Dependent Variables"

#### Quick Transformation and Merger

In [74]:
# Make a new column where we add up male and female
occupation_detailed_df.loc[:, "d_Hauptberuf"] = occupation_detailed_df["d_Hauptberuf_m"] + occupation_detailed_df["d_Hauptberuf_w"]

In [75]:
# Since we only have data on the Regierungsbezirk level we need to find the id

# Merge the DataFrames on the 'id_inequality' column
occupation_detailed_df = pd.merge(occupation_detailed_df, control_df[["id_inequality", "rb_id"]], on="id_inequality", how="left")

# Give new columns for skills
occupation_detailed_df["high"] = ""
occupation_detailed_df["medium"] = ""
occupation_detailed_df["low"] = ""
occupation_detailed_df["unskilled"] = ""
occupation_detailed_df["intermediate"] = 0

# Show the first few rows of the merged DataFrame to confirm the inclusion of 'rb_id'
occupation_detailed_df.head(-5)

,id_inequality,i_Grossbuchstabe,i_Ziffer,d_Hauptberuf_m,d_Hauptberuf_w,d_Hauptberuf,rb_id,high,medium,low,unskilled,intermediate
0,1.0,A,1,6541.0,5786.0,12327.0,17.0,,,,,0
1,1.0,A,2,38.0,10.0,48.0,17.0,,,,,0
2,1.0,A,4,69.0,100.0,169.0,17.0,,,,,0
3,1.0,A,5,397.0,110.0,507.0,17.0,,,,,0
4,1.0,A,6,187.0,36.0,223.0,17.0,,,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...
80879,702.0,E,1,4.0,0.0,4.0,31.0,,,,,0
80880,702.0,E,2,41.0,3.0,44.0,31.0,,,,,0
80881,702.0,E,3,34.0,0.0,34.0,31.0,,,,,0
80882,702.0,E,4,40.0,8.0,48.0,31.0,,,,,0


In [76]:
control_df

,id_inequality,rb_id,rb_name,province_id,s_industry,s_prot,s_german,longitud,latitud,slope_km,soil_suitability,mean_temp,mean_preci,dist_100km,log_distance_carbon_c
0,1.0,17.0,Koenigsberg,1.0,18.515817,93.379268,54.543988,21.261999,55.648117,0.039,3.466667,7.255,64.419,0.330,4.272285
1,2.0,17.0,Koenigsberg,1.0,20.088563,96.752059,100.585163,20.249264,54.843463,0.041,3.272727,7.703,63.562,0.101,5.116106
2,4.0,17.0,Koenigsberg,1.0,35.498406,91.659959,76.561623,20.503550,54.706534,0.041,2.000000,7.983,66.168,0.040,5.181000
3,5.0,17.0,Koenigsberg,1.0,15.060419,97.782661,137.406224,20.646094,54.737281,0.039,2.052632,7.516,65.371,0.009,5.153863
4,6.0,17.0,Koenigsberg,1.0,12.007056,98.668155,89.333490,21.267903,54.855057,0.038,2.050000,7.287,61.743,0.119,5.070227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,698.0,1.0,Aachen,12.0,15.278715,1.613553,64.254487,6.167253,50.364714,0.143,2.785714,7.611,93.091,0.724,3.452736
571,699.0,31.0,Sigmaringen,12.0,19.492555,5.190840,93.931298,9.235305,48.006617,0.165,1.375000,7.392,73.282,0.250,4.136785
572,700.0,31.0,Sigmaringen,12.0,23.974037,2.518404,99.023634,9.198332,48.235600,0.186,1.000000,6.673,78.484,0.419,4.088633
573,701.0,31.0,Sigmaringen,12.0,30.325834,6.910838,88.408480,9.001089,48.317343,0.182,1.000000,7.312,74.583,0.376,3.776254


In [77]:
rb_2_pro = control_df.groupby("province_id")["rb_id"].apply(lambda x: list(set(x))).reset_index()
rb_2_pro

,province_id,rb_id
0,1.0,"[17.0, 11.0]"
1,2.0,"[22.0, 7.0]"
2,3.0,"[10.0, 4.0, 29.0]"
3,4.0,"[33.0, 18.0, 34.0]"
4,5.0,"[28.0, 6.0]"
5,6.0,"[26.0, 19.0, 5.0]"
6,7.0,"[9.0, 21.0, 23.0]"
7,8.0,[30.0]
8,9.0,"[32.0, 3.0, 12.0, 13.0, 20.0, 27.0]"
9,10.0,"[24.0, 25.0, 2.0]"


["Allenstein", "Neidenburg", "Ortelsburg", "Osterode Ostpreussen", "Roessel", 
              "Sensburg", "Johannisburg", "Loetzen", "Lyck"]

In [78]:
# Give counties of newly (in 1907) formed Allenstein its own rb_id and rb_name

counties_Allenstein = [15, 16, 19, 20, 21, 40, 41, 42, 43]
occupation_detailed_df.loc[occupation_detailed_df["id_inequality"].isin(counties_Allenstein) , "rb_id"] = 37

In [79]:
occupation_detailed_df.dtypes, control_df.dtypes

(id_inequality       float64
 i_Grossbuchstabe     object
 i_Ziffer              int16
 d_Hauptberuf_m      float64
 d_Hauptberuf_w      float64
 d_Hauptberuf        float64
 rb_id               float64
 high                 object
 medium               object
 low                  object
 unskilled            object
 intermediate          int64
 dtype: object,
 id_inequality            float64
 rb_id                    float64
 rb_name                   object
 province_id              float64
 s_industry               float64
 s_prot                   float64
 s_german                 float64
 longitud                 float64
 latitud                  float64
 slope_km                 float64
 soil_suitability         float64
 mean_temp                float64
 mean_preci               float64
 dist_100km               float64
 log_distance_carbon_c    float64
 dtype: object)

# Prepare dictionary

In [80]:
# Read the HISCLASS.do file and extract lines
with open(f"{classification_path}\HISCLASS.do") as file:
    lines = file.readlines()

# Make a dictionary
# Regular expression to capture HISCO code and its class
pattern = r"\((\d{5})=(\d+)\)"

# Extract all the mappings from the lines
mappings = {}
for line in lines:
    matches = re.findall(pattern, line)
    for hisco_code, hisco_class in matches:
        mappings[hisco_code] = int(hisco_class)

# Convert all values in the mappings dictionary to strings
mappings = {key: str(value) for key, value in mappings.items()}

mappings

{'99997': '1',
 '99998': '1',
 '01110': '2',
 '01120': '2',
 '01130': '2',
 '01140': '2',
 '01150': '2',
 '01190': '2',
 '01210': '2',
 '01220': '2',
 '01230': '2',
 '01240': '2',
 '01250': '2',
 '01260': '2',
 '01270': '2',
 '01280': '2',
 '01290': '2',
 '01320': '2',
 '01330': '2',
 '01340': '2',
 '01350': '2',
 '01390': '2',
 '01400': '4',
 '01420': '4',
 '01430': '4',
 '01490': '4',
 '02000': '2',
 '02120': '2',
 '02130': '2',
 '02140': '2',
 '02210': '2',
 '02220': '2',
 '02230': '2',
 '02235': '2',
 '02240': '2',
 '02245': '2',
 '02250': '2',
 '02255': '2',
 '02260': '4',
 '02290': '2',
 '02305': '2',
 '02310': '2',
 '02320': '2',
 '02330': '2',
 '02340': '2',
 '02390': '2',
 '02410': '2',
 '02420': '2',
 '02430': '2',
 '02440': '2',
 '02450': '2',
 '02460': '2',
 '02470': '2',
 '02480': '2',
 '02485': '2',
 '02490': '2',
 '02510': '2',
 '02520': '2',
 '02590': '2',
 '02620': '2',
 '02630': '2',
 '02690': '2',
 '02710': '2',
 '02720': '2',
 '02730': '2',
 '02740': '2',
 '02790': 

In [81]:
# Add the new HISCO title for the "Intermediate"
mappings["66666"] = "13"

In [82]:
value_exists_in_mappings = "66666" in mappings

mapping_value = mappings.get("66666", "Not found")

value_exists_in_mappings, mapping_value

(True, '13')

In [83]:
# Create a mapping dictionary for the Skills
skill_mappings = {
    "1": "high",
    "2": "high",
    "3": "medium",
    "4": "medium",
    "5": "low",
    "6": "medium",
    "7": "medium",
    "8": "medium",
    "9": "low",
    "10": "low",
    "11": "unskilled",
    "12": "unskilled",
    "13": "medium"} #for occupations with multiple skill levels

# Helper Functions

In [84]:
def replace_values_with_mappings(df, columns, mappings):

    """
    Replace the values in the specified columns of the DataFrame using the provided mappings dictionary.
    Also report any values that could not be replaced and their positions.
    Parameters:
    df (DataFrame): The DataFrame to modify
    columns (list): The list of columns to replace values in
    mappings (dict): The dictionary to use for the replacements

    Returns:
    DataFrame: The DataFrame with replaced values
    list: A list of tuples containing column names and index positions where values could not be replaced
    """
    not_replaced = []

    # Loop over each column

    for column in columns:
        # Loop over each row in the column
        for idx, value in enumerate(df[column]):
            try:
                # Try to replace the value using the mappings dictionary
                df.loc[idx, column] = mappings[value]
            except KeyError:
                # If the value is not in the mappings dictionary, record its position
                not_replaced.append((column, idx))          

    return df, not_replaced

In [85]:
def SkillCalculator(df_agri, df_A_Rest, status_cols, HISCO_cols, threshold_criteria = "Average_Firm_Size", threshold_lvl = 50):
    
    """
    This function performs several operations to update df_A_Rest based on the values in the DataFrame df_agri.
    The purpose is to use the distributional data on the status (e.g. uneducated laborer) and the corresponding
    HISCO, which was assigned by Hand, for each "Berufszweig" in the DataFrame df_agri.   
    1. It first calculates "_per" columns in the DataFrame, which are the proportions of each column
       related to the total sum of specific columns.
    2. The function then uses a threshold from the "Average_Firm_Size" column to decide how to 
       update df_A_Rest. If the average firm size is less than 50, it performs specific calculations
       to update skill levels ("high", "medium", "low", "unskilled") in df_A_Rest.

    Parameters:
    df_agri (DataFrame): Source DataFrame containing classifications and shares of occupations per profession.
    df_A_Rest (DataFrame): Orginal occupational data. Target DataFrame to be updated based on df_agri.

    Returns:
    df_agri (DataFrame): Updated source DataFrame with additional "_per" columns.
    df_A_Rest (DataFrame): Updated target DataFrame with skill level columns.
    """
    
    # Step 1: Create new "_per" columns
    cols_to_sum = status_cols
    df_agri["total"] = df_agri[cols_to_sum].sum(axis=1)
    for col in cols_to_sum:
        new_col_name = f"{col}_per"
        df_agri[new_col_name] = df_agri[col] / df_agri["total"]

    # Step 2: Convert "Average_Firm_Size" to float
    df_agri[threshold_criteria] = df_agri[threshold_criteria].astype(float)
    

    # Initialize empty columns in df_A_Rest
    df_A_Rest.loc[:, "high"] = 0
    df_A_Rest.loc[:, "medium"] = 0
    df_A_Rest.loc[:, "low"] = 0
    df_A_Rest.loc[:, "unskilled"] = 0

    # Step 3: Loop through each row in df_agri
    for idx, row in df_agri.iterrows():
        average_firm_size = row[threshold_criteria]
        special_case = row["Eindeutig_Identifizierbar"] == "Sonderfall"
        if average_firm_size < threshold_lvl and not special_case:
            # Calculate Handwerker
            handwerker_cols = [f"{col}_per" for col in cols_to_sum if col not in ["c3", "c1"]]
            handwerker = row[handwerker_cols].sum()
            print(f"Row {idx}: Calculated Handwerker as {handwerker} using columns {handwerker_cols}")
            
            # Calculate no Handwerker
            no_handwerker_cols = [f"{col}_per" for col in cols_to_sum if col in ["c3", "c1"]]
            no_handwerker = row[no_handwerker_cols].sum()
            print(f"Row {idx}: Calculated No_Handwerker as {no_handwerker} using columns {no_handwerker_cols}")
            
            # Find corresponding row(s) in df_A_Rest
            mask = (df_A_Rest["i_Grossbuchstabe"] == row["Letter"]) & (df_A_Rest["i_Ziffer"] == row["Number"])
            df_target_rows = df_A_Rest[mask]

            # Update df_A_Rest
            for target_idx, target_row in df_target_rows.iterrows():
                value_to_add = handwerker * target_row["d_Hauptberuf"]
                skill_level_c2 = row["c2_HISCO"]
                skill_level_c3 = row["c3_HISCO"]

                value_to_add_c3 = no_handwerker * target_row["d_Hauptberuf"]

                # Sum up the values if both c2_HISCO and c3_HISCO contain the same value
                df_A_Rest.loc[target_idx, skill_level_c2] += value_to_add
                df_A_Rest.loc[target_idx, skill_level_c3] += value_to_add_c3
        
         # New case for Average_Firm_Size >= 50

        else:
            # Find corresponding row(s) in df_A_Rest
            mask = (df_A_Rest["i_Grossbuchstabe"] == row["Letter"]) & (df_A_Rest["i_Ziffer"] == row["Number"])
            df_target_rows = df_A_Rest[mask]        

            # Columns to consider from a1 to c3, excluding c1
            cols_to_consider = HISCO_cols            

            # Update df_A_Rest based on these columns
            for target_idx, target_row in df_target_rows.iterrows():
                for col in cols_to_consider:
                    col_per = f"{col}_per"
                    skill_level = row[f"{col}_HISCO"]                   

                    # Special case: Include c1 as part of c2
                    if col == "c3":
                        value_to_add = (row["c3_per"] + row["c1_per"]) * target_row["d_Hauptberuf"]
                    else:
                        value_to_add = row[col_per] * target_row["d_Hauptberuf"]                        

                    df_A_Rest.loc[target_idx, skill_level] += value_to_add

    return df_agri, df_A_Rest

# Agriculture

In [86]:
# Filter the data so we only have the people working in agriculture
# First, filter rows where "i_Grossbuchstabe" == "A"

df_A = occupation_detailed_df[occupation_detailed_df["i_Grossbuchstabe"] == "A"]

# Then, from this filtered DataFrame, filter rows where "i_Ziffer" == 1

df_A_1 = df_A[df_A["i_Ziffer"] == 1]

df_A_Rest = occupation_detailed_df[(occupation_detailed_df["i_Grossbuchstabe"] == "A") & (occupation_detailed_df["i_Ziffer"].isin([2, 3, 5, 6]))]

# Display the final filtered DataFrame

df_A_1

,id_inequality,i_Grossbuchstabe,i_Ziffer,d_Hauptberuf_m,d_Hauptberuf_w,d_Hauptberuf,rb_id,high,medium,low,unskilled,intermediate
0,1.0,A,1,6541.0,5786.0,12327.0,17.0,,,,,0
151,2.0,A,1,6958.0,2642.0,9600.0,17.0,,,,,0
278,4.0,A,1,775.0,541.0,1316.0,17.0,,,,,0
470,5.0,A,1,7888.0,4313.0,12201.0,17.0,,,,,0
580,6.0,A,1,8346.0,6940.0,15286.0,17.0,,,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...
80309,698.0,A,1,6929.0,4864.0,11793.0,1.0,,,,,0
80437,699.0,A,1,3807.0,3802.0,7609.0,31.0,,,,,0
80564,700.0,A,1,2647.0,300.0,2947.0,31.0,,,,,0
80672,701.0,A,1,3313.0,3884.0,7197.0,31.0,,,,,0


## **1. Agriculture** (df_A_1)

In [87]:
Agri_rb = pd.read_excel(f"{classification_path}\Agriculture_Regionalized.xlsx", dtype=str)
Agri_rb

,Regierungsbezirk,id,province,province_name,a1,a2,b,c,Summe,Fläche der Betriebe mit über 100ha
0,Aachen,1,12,Rheinprovinz,11414.10965504654,17162.89034495346,188,54922,83687,4.4
1,Allenstein,37,1,Ostpreußen,22358.079997615572,10659.92000238443,1157,107974,142149,30.8
2,Arnsberg,2,10,Westfalen,15318.370995750969,13712.629004249031,646,76458,106135,4.3
3,Aurich,3,9,Hannover,7328.028000770663,8724.971999229336,209,44926,61188,3.8
4,Breslau,5,6,Schlesien,33898.66478795318,24788.33521204682,4705,234295,297687,41.6
5,Bromberg,6,5,Posen,23026.96644018288,8972.033559817119,2593,142645,177237,44.4
6,Danzig,7,2,Westpreußen,17593.065254512927,7775.93474548707,1646,97352,124367,31.9
7,Düsseldorf,8,12,Rheinprovinz,16783.36947200784,17769.630527992158,570,92568,127691,3.1
8,Erfurt,9,7,Sachsen,10256.883651487682,8561.116348512318,436,52277,71531,15.6
9,Frankfurt,10,3,Brandenburg,33396.12918660287,24323.87081339713,2676,210002,270398,35.3


==> percentages für a1 (General Farmers, medium skilled) und a2 (Subistenzlandwirtschaft, low skilled)

In [ ]:
#calculate the percentages

# for a1
Agri_rb["a1"] = Agri_rb["a1"].astype(float)
Agri_rb["Summe"] = Agri_rb["Summe"].astype(float)
Agri_rb["a1_per"] = Agri_rb["a1"] / Agri_rb["Summe"]

# for a2
Agri_rb["a2"] = Agri_rb["a2"].astype(float)
Agri_rb["Summe"] = Agri_rb["Summe"].astype(float)
Agri_rb["a2_per"] = Agri_rb["a2"] / Agri_rb["Summe"]

# for b
Agri_rb["b"] = Agri_rb["b"].astype(float)
Agri_rb["Summe"] = Agri_rb["Summe"].astype(float)
Agri_rb["b_per"] = Agri_rb["b"] / Agri_rb["Summe"]

# for c
Agri_rb["c"] = Agri_rb["c"].astype(float)
Agri_rb["Summe"] = Agri_rb["Summe"].astype(float)
Agri_rb["c_per"] = Agri_rb["c"] / Agri_rb["Summe"]

In [ ]:
Agri_rb

In [ ]:
# Convert 'rb_id' in both DataFrames to string to ensure they have the same format

df_A_1["rb_id"] = df_A_1["rb_id"].astype(str)

Agri_rb["id"] = Agri_rb["id"].astype(str)

# Remove the '.0' from the 'rb_id' values in df_A_1
df_A_1["rb_id"] = df_A_1["rb_id"].str.replace('.0', '', regex=False)

# Perform the merge 
df_A_1 = pd.merge(df_A_1, Agri_rb[["id", "a1_per", "a2_per", "b_per", "c_per"]], left_on="rb_id", right_on="id", how="left")

# Show some rows
df_A_1.head()

In [ ]:
# Convert 'a_per', 'b_per', 'c_per', and 'd_Hauptberuf_total' to float for calculations

df_A_1["a1_per"] = df_A_1["a1_per"].astype(float)
df_A_1["a2_per"] = df_A_1["a2_per"].astype(float)
df_A_1["b_per"] = df_A_1["b_per"].astype(float)
df_A_1["c_per"] = df_A_1["c_per"].astype(float)
df_A_1["d_Hauptberuf"] = df_A_1["d_Hauptberuf"].astype(float)

# Create and fill the 'medium' and 'unskilled' columns

df_A_1["high"] = ''
df_A_1["medium"] = df_A_1["d_Hauptberuf"] * df_A_1["a1_per"]
df_A_1["low"] = df_A_1["d_Hauptberuf"] * (df_A_1["a2_per"] + df_A_1["b_per"])
df_A_1["unskilled"] = df_A_1["d_Hauptberuf"] * df_A_1["c_per"]

# Reorder the columns

column_order = ["id_inequality", "i_Grossbuchstabe", "i_Ziffer", "d_Hauptberuf_m", "d_Hauptberuf_w", "rb_id", "d_Hauptberuf", "high", "medium", "low", "unskilled"]

df_A_1 = df_A_1[column_order]
df_A_1

## Other Primary Professions (df_A_Rest)

In [ ]:
df_A_Rest

In [ ]:
# Load the data to classify Agri_Rest

df_agri_rest = pd.read_excel(f"{classification_path}\Intermediate_Primary_Sector_Rest.xlsx", dtype=str)

In [ ]:
# Columns to replace values in
columns_to_replace = ["a1_HISCO", "a2_HISCO", "a3_HISCO", "b1_HISCO", "b2_HISCO", "b3_HISCO", "c2_HISCO", "c3_HISCO"]

# Use the function to replace values in df_agri_rest
df_agri_rest, not_replaced_positions = replace_values_with_mappings(df_agri_rest, columns_to_replace, mappings)

# Show the first few rows of the modified DataFrame and the positions where values could not be replaced
df_agri_rest, not_replaced_positions

In [ ]:
pd.set_option("display.max_columns", None)
df_agri_rest

In [ ]:
df_agri_rest, not_replaced_positions = replace_values_with_mappings(df_agri_rest, columns_to_replace, skill_mappings)
df_agri_rest

In [ ]:
cols_to_check = ["a1", "a2", "a3", "b1", "b2", "b3", "c1", "c2", "c3", "afr", "Average_Firm_Size"]
print(df_agri_rest[cols_to_check].dtypes)
print(df_A_Rest["d_Hauptberuf"].dtypes)

# Convert to floats
df_agri_rest[cols_to_check] = df_agri_rest[cols_to_check].apply(pd.to_numeric, errors="coerce")

# Convert 'd_Hauptberuf' column in df_A_Rest to float
df_A_Rest["d_Hauptberuf"] = pd.to_numeric(df_A_Rest["d_Hauptberuf"], errors="coerce")

# Check the data types after conversion
agri_dtypes = df_agri_rest[cols_to_check].dtypes
a_rest_dtype = df_A_Rest["d_Hauptberuf"].dtype
print(agri_dtypes)
print(a_rest_dtype)

In [ ]:
# Define the columns
#df_agri_rest["c3"] = df_agri_rest["c3"] + df_agri_rest["afr"]

# HISCO cols
cols_to_consider = ["a1", "a2", "a3", "b1", "b2", "b3", "c2", "c3"]

# columns to consider
status_cols = ["a1", "a2", "a3", "b1", "b2", "b3", "c1", "c2", "c3"]
df_A_Rest.loc[:, "i_Ziffer"] = df_A_Rest["i_Ziffer"].astype(str)
df_agri_rest_filled3, df_A_Rest = SkillCalculator(df_agri_rest, df_A_Rest, status_cols, cols_to_consider)
df_A_Rest

# Manufacturing and Crafts

In [ ]:
# Filter the data so we only have the people working in industry
# First, filter rows where "i_Grossbuchstabe" == "B"

df_Ind = occupation_detailed_df[occupation_detailed_df["i_Grossbuchstabe"] == "B"]

# Drop people in the industrial sector that couldn't be assigned

df_Ind = df_Ind[df_Ind["i_Ziffer"] != 166]

# Display the final filtered DataFrame

df_Ind

In [ ]:
# Load the data to classify df_Ind
df_dist_Ind = pd.read_excel(f"{classification_path}\Intermediate_Manufacturing_and_Crafts.xlsx", dtype=str)
df_dist_Ind = df_dist_Ind[df_dist_Ind["Number"] != "166"]
df_dist_Ind

In [ ]:
# Columns to replace values in
columns_to_replace = ["a1_HISCO", "a2_HISCO", "a3_HISCO", "b1_HISCO", "b2_HISCO", "b3_HISCO", "c2_HISCO", "c3_HISCO"]

# Use the function to replace values in df_agri_rest
df_dist_Ind, not_replaced_positions = replace_values_with_mappings(df_dist_Ind, columns_to_replace, mappings)

# Show the first few rows of the modified DataFrame and the positions where values could not be replaced
df_dist_Ind, not_replaced_positions

In [ ]:
# Now map the to skill level
df_dist_Ind, not_replaced_positions = replace_values_with_mappings(df_dist_Ind, columns_to_replace, skill_mappings)
df_dist_Ind

In [ ]:
cols_to_check = ["a1", "a2", "a3", "b1", "b2", "b3", "c1", "c2", "c3", "afr", "Average_Firm_Size"]
print(df_dist_Ind[cols_to_check].dtypes)
print(df_Ind["d_Hauptberuf"].dtypes)

# Convert to floats
df_dist_Ind[cols_to_check] = df_dist_Ind[cols_to_check].apply(pd.to_numeric, errors='coerce')

# Convert 'd_Hauptberuf' column in df_Ind to float
df_Ind["d_Hauptberuf"] = pd.to_numeric(df_Ind["d_Hauptberuf"], errors='coerce')

# Check the data types after conversion
dist_Ind_dtypes = df_dist_Ind[cols_to_check].dtypes
Ind_dtype = df_Ind["d_Hauptberuf"].dtype
print(dist_Ind_dtypes)
print(Ind_dtype)

In [ ]:
# Define the columns
status_cols = ["a1", "a2", "a3", "b1", "b2", "b3", "c1", "c2", "c3"]
cols_to_consider = ["a1", "a2", "a3", "b1", "b2", "b3", "c2", "c3"]

df_Ind.loc[:, "i_Ziffer"] = df_Ind["i_Ziffer"].astype(str)
df_dist_Ind_finished, df_Ind = SkillCalculator(df_dist_Ind, df_Ind, status_cols, cols_to_consider)
df_Ind

# Services

In [ ]:
# Filter the data so we only have the people working in industry
# First, filter rows where "i_Grossbuchstabe" == "C", "D" or "E"

df_Serv_C = occupation_detailed_df[occupation_detailed_df["i_Grossbuchstabe"].isin(["C", "D"])]

# Drop people in the industrial sector that couldn't be assigned

df_Serv_C = df_Serv_C[~df_Serv_C["i_Ziffer"].isin([12, 13])]

# Display the final filtered DataFrame

df_Serv_E = occupation_detailed_df[occupation_detailed_df["i_Grossbuchstabe"].isin(["E"])]

# Drop people in the industrial sector that couldn't be assigned

df_Serv_E = df_Serv_E[~df_Serv_E["i_Ziffer"].isin([1, 2, 3, 4, 5, 8])]

# Merge both DataFrames

df_Ser = pd.concat([df_Serv_C, df_Serv_E], ignore_index=True)

# Let's have a look

df_Ser[:-40]

In [ ]:
# Load the data to classify df_Ind
df_dist_Ser = pd.read_excel(f"{classification_path}\Intermediate_Services.xlsx", dtype=str)
df_dist_Ser

In [ ]:
# Columns to replace values in
columns_to_replace = ["a1_HISCO", "a2_HISCO", "a3_HISCO", "b_HISCO", "c2_HISCO", "c3_HISCO"]

# Use the function to replace values in df_agri_rest
df_dist_Ser, not_replaced_positions = replace_values_with_mappings(df_dist_Ser, columns_to_replace, mappings)

# Show the first few rows of the modified DataFrame and the positions where values could not be replaced
df_dist_Ser, not_replaced_positions

In [ ]:
# Now map the to skill level
df_dist_Ser, not_replaced_positions = replace_values_with_mappings(df_dist_Ser, columns_to_replace, skill_mappings)
df_dist_Ser

In [ ]:
cols_to_check = ["a1", "a2", "a3", "b", "c1", "c2", "c3", "Average_Firm_Size"]
print(df_dist_Ser[cols_to_check].dtypes)
print(df_Ser["d_Hauptberuf"].dtypes)

# Convert to floats
df_dist_Ser[cols_to_check] = df_dist_Ser[cols_to_check].apply(pd.to_numeric, errors="coerce")

# Convert 'd_Hauptberuf' column in df_A_Rest to float
df_Ser["d_Hauptberuf"] = pd.to_numeric(df_Ser["d_Hauptberuf"], errors="coerce")

# Check the data types after conversion
dist_Ser_dtypes = df_dist_Ser[cols_to_check].dtypes
Ser_dtype = df_Ser["d_Hauptberuf"].dtype
print(dist_Ser_dtypes)
print(Ser_dtype)

In [ ]:
# Define the columns
status_cols = ["a1", "a2", "a3", "b", "c1", "c2", "c3"]
cols_to_consider = ["a1", "a2", "a3", "b", "c2", "c3"]
df_Ser.loc[:, "i_Ziffer"] = df_Ser["i_Ziffer"].astype(str)
df_dist_Ser_finished, df_Ser_finished = SkillCalculator(df_dist_Ser, df_Ser, status_cols, cols_to_consider, threshold_lvl=1)
df_Ser_finished

# Special

## Forestry

In [ ]:
df_Forestry = occupation_detailed_df[occupation_detailed_df["i_Grossbuchstabe"].isin(["A"])]
df_Forestry = df_Forestry[df_Forestry["i_Ziffer"].isin([4])]

In [ ]:
# Load data
Forestry_rb = pd.read_excel(f"{classification_path}\\Forestry.xlsx")

# Calculate the percentages

# for a1
Forestry_rb["a"] = Forestry_rb["a"].astype(float)
Forestry_rb["Summe"] = Forestry_rb["Summe"].astype(float)
Forestry_rb["a_per"] = Forestry_rb["a"] / Forestry_rb["Summe"]

# for b
Forestry_rb["b"] = Forestry_rb["b"].astype(float)
Forestry_rb["Summe"] = Forestry_rb["Summe"].astype(float)
Forestry_rb["b_per"] = Forestry_rb["b"] / Forestry_rb["Summe"]

# for c
Forestry_rb["c"] = Forestry_rb["c"].astype(float)
Forestry_rb["Summe"] = Forestry_rb["Summe"].astype(float)
Forestry_rb["c_per"] = Forestry_rb["c"] / Forestry_rb["Summe"]

# Convert 'rb_id' in both DataFrames to string to ensure they have the same format
df_Forestry["rb_id"] = df_Forestry["rb_id"].astype(str)
Forestry_rb["id"] = Forestry_rb["id"].astype(str)

# Remove the '.0' suffix from the 'rb_id' values in df_A_1
df_Forestry["rb_id"] = df_Forestry["rb_id"].str.replace(".0", "", regex=False)

# Perform the merge
df_Forestry = pd.merge(df_Forestry, Forestry_rb[["id", "a_per", "b_per", "c_per"]], left_on="rb_id", right_on="id", how="left")

# Convert 'a_per', 'b_per', 'c_per', and 'd_Hauptberuf' to float for calculations
df_Forestry["a_per"] = df_Forestry["a_per"].astype(float)
df_Forestry["b_per"] = df_Forestry["b_per"].astype(float)
df_Forestry["c_per"] = df_Forestry["c_per"].astype(float)
df_Forestry["d_Hauptberuf"] = df_Forestry["d_Hauptberuf"].astype(float)

# Create and fill the 'medium' and 'unskilled' columns
df_Forestry["high"] = ""
# a and b are medium skilled (HISCO: 63220)
df_Forestry["medium"] = df_Forestry["d_Hauptberuf"] * (df_Forestry["a_per"] + df_Forestry["b_per"])
df_Forestry["low"] = df_Forestry["d_Hauptberuf"] * df_Forestry["c_per"]
df_Forestry["unskilled"] = ""

# Reorder the columns
column_order = ["id_inequality", "i_Grossbuchstabe", "i_Ziffer", "d_Hauptberuf_m", "d_Hauptberuf_w", "rb_id", "d_Hauptberuf", "high", "medium", "low", "unskilled"]
df_Forestry = df_Forestry[column_order]
df_Forestry


## Post and Telegram

In [ ]:
df_Post = occupation_detailed_df[occupation_detailed_df["i_Grossbuchstabe"].isin(["C"])]
df_Post = df_Post[df_Post["i_Ziffer"].isin([12])]

In [ ]:
# Load data
Post_rb = pd.read_excel(f"{classification_path}\\Post_and_Telegram.xlsx")

# Calculate the percentages

# For a1
Post_rb["a"] = Post_rb["a"].astype(float)
Post_rb["Summe"] = Post_rb["Summe"].astype(float)
Post_rb["a_per"] = Post_rb["a"] / Post_rb["Summe"]

# For b
Post_rb["b"] = Post_rb["b"].astype(float)
Post_rb["Summe"] = Post_rb["Summe"].astype(float)
Post_rb["b_per"] = Post_rb["b"] / Post_rb["Summe"]

# For c
Post_rb["c"] = Post_rb["c"].astype(float)
Post_rb["Summe"] = Post_rb["Summe"].astype(float)
Post_rb["c_per"] = Post_rb["c"] / Post_rb["Summe"]

# Convert 'rb_id' in both DataFrames to string to ensure they have the same format
df_Post["rb_id"] = df_Post["rb_id"].astype(str)
Post_rb["id"] = Post_rb["id"].astype(str)

# Remove the '.0' suffix from the 'rb_id' values in df_A_1
df_Post["rb_id"] = df_Post["rb_id"].str.replace(".0", "", regex=False)

# Perform the merge
df_Post = pd.merge(df_Post, Post_rb[["id", "a_per", "b_per", "c_per"]], left_on="rb_id", right_on="id", how="left")

# Convert 'a_per', 'b_per', 'c_per', and 'd_Hauptberuf' to float for calculations
df_Post["a_per"] = df_Post["a_per"].astype(float)
df_Post["b_per"] = df_Post["b_per"].astype(float)
df_Post["c_per"] = df_Post["c_per"].astype(float)
df_Post["d_Hauptberuf"] = df_Post["d_Hauptberuf"].astype(float)

# Create and fill the 'medium' and 'unskilled' columns

# a is highly skilled (HISCO: 21110)
df_Post["high"] = df_Post["d_Hauptberuf"] * df_Post["a_per"]
df_Post["medium"] = ""

# b and are low skilled (HISCO: 39310 and 37000)
df_Post["low"] = df_Post["d_Hauptberuf"] * (df_Post["b_per"] + df_Post["c_per"])
df_Post["unskilled"] = ""

# Reorder the columns
column_order = ["id_inequality", "i_Grossbuchstabe", "i_Ziffer", "d_Hauptberuf_m", "d_Hauptberuf_w", "rb_id", "d_Hauptberuf", "high", "medium", "low", "unskilled"]
df_Post = df_Post[column_order]
df_Post

## Railroad

In [ ]:
df_Rail = occupation_detailed_df[occupation_detailed_df["i_Grossbuchstabe"].isin(["C"])]
df_Rail = df_Rail[df_Rail["i_Ziffer"].isin([13])]

In [ ]:
# Load data
Rail_rb = pd.read_excel(f"{classification_path}\\Railroad.xlsx")

# Calculate the percentages

# For a1
Rail_rb["a"] = Rail_rb["a"].astype(float)
Rail_rb["Summe"] = Rail_rb["Summe"].astype(float)
Rail_rb["a_per"] = Rail_rb["a"] / Rail_rb["Summe"]

# For b
Rail_rb["b"] = Rail_rb["b"].astype(float)
Rail_rb["Summe"] = Rail_rb["Summe"].astype(float)
Rail_rb["b_per"] = Rail_rb["b"] / Rail_rb["Summe"]

# For c
Rail_rb["c"] = Rail_rb["c"].astype(float)
Rail_rb["Summe"] = Rail_rb["Summe"].astype(float)
Rail_rb["c_per"] = Rail_rb["c"] / Rail_rb["Summe"]

# Convert 'rb_id' in both DataFrames to string to ensure they have the same format
df_Rail["rb_id"] = df_Rail["rb_id"].astype(str)
Rail_rb["id"] = Rail_rb["id"].astype(str)

# Remove the '.0' suffix from the 'rb_id' values in df_A_1
df_Rail["rb_id"] = df_Rail["rb_id"].str.replace(".0", "", regex=False)

# Perform the merge
df_Rail = pd.merge(df_Rail, Rail_rb[["id", "a_per", "b_per", "c_per"]], left_on="rb_id", right_on="id", how="left")

# Convert 'a_per', 'b_per', 'c_per', and 'd_Hauptberuf' to float for calculations
df_Rail["a_per"] = df_Rail["a_per"].astype(float)
df_Rail["b_per"] = df_Rail["b_per"].astype(float)
df_Rail["c_per"] = df_Rail["c_per"].astype(float)
df_Rail["d_Hauptberuf"] = df_Rail["d_Hauptberuf"].astype(float)

# Create and fill the 'medium' and 'unskilled' columns

# a is highly skilled (HISCO: 21110)
df_Rail["high"] = df_Rail["d_Hauptberuf"] * df_Rail["a_per"]
df_Rail["medium"] = ""

# b and c are low skilled (HISCO: 39310, 36020)
df_Rail["low"] = df_Rail["d_Hauptberuf"] * (df_Rail["b_per"] + df_Rail["c_per"])
df_Rail["unskilled"] = ""

# Reorder the columns
column_order = ["id_inequality", "i_Grossbuchstabe", "i_Ziffer", "d_Hauptberuf_m", "d_Hauptberuf_w", "rb_id", "d_Hauptberuf", "high", "medium", "low", "unskilled"]
df_Rail = df_Rail[column_order]
df_Rail

## Army

In [ ]:
df_Army = occupation_detailed_df[occupation_detailed_df["i_Grossbuchstabe"].isin(["E"])]
df_Army = df_Army[df_Army["i_Ziffer"].isin([1])]

In [ ]:
# Load data
Army_rb = pd.read_excel(f"{classification_path}\\Military.xlsx")

# Calculate the percentages

# For a1
Army_rb["a"] = Army_rb["a"].astype(float)
Army_rb["Summe"] = Army_rb["Summe"].astype(float)
Army_rb["a_per"] = Army_rb["a"] / Army_rb["Summe"]

# For b
Army_rb["b"] = Army_rb["b"].astype(float)
Army_rb["Summe"] = Army_rb["Summe"].astype(float)
Army_rb["b_per"] = Army_rb["b"] / Army_rb["Summe"]

# Convert 'rb_id' in both DataFrames to string to ensure they have the same format
df_Army["rb_id"] = df_Army["rb_id"].astype(str)
Army_rb["id"] = Army_rb["id"].astype(str)

# Remove the '.0' suffix from the 'rb_id' values in df_A_1
df_Army["rb_id"] = df_Army["rb_id"].str.replace(".0", "", regex=False)

# Perform the merge
df_Army = pd.merge(df_Army, Army_rb[["id", "a_per", "b_per"]], left_on="rb_id", right_on="id", how="left")

# Convert 'a_per', 'b_per', 'c_per', and 'd_Hauptberuf' to float for calculations
df_Army["a_per"] = df_Army["a_per"].astype(float)
df_Army["b_per"] = df_Army["b_per"].astype(float)
df_Army["d_Hauptberuf"] = df_Army["d_Hauptberuf"].astype(float)

# Create and fill the 'medium' and 'unskilled' columns

# a is highly skilled (HISCO: 58320)
df_Army["high"] = df_Army["d_Hauptberuf"] * df_Army["a_per"]
df_Army["medium"] = ""
# b is low skilled (HISCO: 58340)
df_Army["low"] = df_Army["d_Hauptberuf"] * df_Army["b_per"]
df_Army["unskilled"] = ""

# Reorder the columns
column_order = ["id_inequality", "i_Grossbuchstabe", "i_Ziffer", "d_Hauptberuf_m", "d_Hauptberuf_w", "rb_id", "d_Hauptberuf", "high", "medium", "low", "unskilled"]
df_Army = df_Army[column_order]
df_Army

## Government

In [ ]:
df_Gov = occupation_detailed_df[occupation_detailed_df["i_Grossbuchstabe"].isin(["E"])]
df_Gov = df_Gov[df_Gov["i_Ziffer"].isin([2])]

In [ ]:
# Load data
Gov_rb = pd.read_excel(f"{classification_path}\\State.xlsx")

# Calculate the percentages

# For a1
Gov_rb["a"] = Gov_rb["a"].astype(float)
Gov_rb["Summe"] = Gov_rb["Summe"].astype(float)
Gov_rb["a_per"] = Gov_rb["a"] / Gov_rb["Summe"]

# For b
Gov_rb["b"] = Gov_rb["b"].astype(float)
Gov_rb["Summe"] = Gov_rb["Summe"].astype(float)
Gov_rb["b_per"] = Gov_rb["b"] / Gov_rb["Summe"]

# For c
Gov_rb["c"] = Gov_rb["c"].astype(float)
Gov_rb["Summe"] = Gov_rb["Summe"].astype(float)
Gov_rb["c_per"] = Gov_rb["c"] / Gov_rb["Summe"]

# Convert 'rb_id' in both DataFrames to string to ensure they have the same format
df_Gov["rb_id"] = df_Gov["rb_id"].astype(str)
Gov_rb["id"] = Gov_rb["id"].astype(str)

# Remove the '.0' suffix from the 'rb_id' values in df_A_1
df_Gov["rb_id"] = df_Gov["rb_id"].str.replace(".0", "", regex=False)

# Perform the merge
df_Gov = pd.merge(df_Gov, Gov_rb[["id", "a_per", "b_per", "c_per"]], left_on="rb_id", right_on="id", how="left")

# Convert 'a_per', 'b_per', 'c_per', and 'd_Hauptberuf' to float for calculations
df_Gov["a_per"] = df_Gov["a_per"].astype(float)
df_Gov["b_per"] = df_Gov["b_per"].astype(float)
df_Gov["c_per"] = df_Gov["c_per"].astype(float)
df_Gov["d_Hauptberuf"] = df_Gov["d_Hauptberuf"].astype(float)

# Create and fill the 'medium' and 'unskilled' columns

# a is highly skilled (HISCO: 20210)
df_Gov["high"] = df_Gov["d_Hauptberuf"] * df_Gov["a_per"]
df_Gov["medium"] = ""

# b and c are low skilled (HISCO: 39310, 59990)
df_Gov["low"] = df_Gov["d_Hauptberuf"] * (df_Gov["b_per"] + df_Gov["c_per"])
df_Gov["unskilled"] = ""

# Reorder the columns
column_order = ["id_inequality", "i_Grossbuchstabe", "i_Ziffer", "d_Hauptberuf_m", "d_Hauptberuf_w", "rb_id", "d_Hauptberuf", "high", "medium", "low", "unskilled"]
df_Gov = df_Gov[column_order]
df_Gov

## Church

In [ ]:
df_Church = occupation_detailed_df[occupation_detailed_df["i_Grossbuchstabe"].isin(["E"])]
df_Church = df_Church[df_Church["i_Ziffer"].isin([3])]

In [ ]:
# Load data
Church_rb = pd.read_excel(f"{classification_path}\\Church.xlsx")

# Calculate the percentages

# For a1
Church_rb["a"] = Church_rb["a"].astype(float)
Church_rb["Summe"] = Church_rb["Summe"].astype(float)
Church_rb["a_per"] = Church_rb["a"] / Church_rb["Summe"]

# For b
Church_rb["b"] = Church_rb["b"].astype(float)
Church_rb["Summe"] = Church_rb["Summe"].astype(float)
Church_rb["b_per"] = Church_rb["b"] / Church_rb["Summe"]

# For c
Church_rb["c"] = Church_rb["c"].astype(float)
Church_rb["Summe"] = Church_rb["Summe"].astype(float)
Church_rb["c_per"] = Church_rb["c"] / Church_rb["Summe"]

# Convert 'rb_id' in both DataFrames to string to ensure they have the same format
df_Church["rb_id"] = df_Church["rb_id"].astype(str)
Church_rb["id"] = Church_rb["id"].astype(str)

# Remove the '.0' suffix from the 'rb_id' values in df_A_1
df_Church["rb_id"] = df_Church["rb_id"].str.replace(".0", "", regex=False)

# Perform the merge
df_Church = pd.merge(df_Church, Church_rb[["id", "a_per", "b_per", "c_per"]], left_on="rb_id", right_on="id", how="left")

# Convert 'a_per', 'b_per', 'c_per', and 'd_Hauptberuf' to float for calculations
df_Church["a_per"] = df_Church["a_per"].astype(float)
df_Church["b_per"] = df_Church["b_per"].astype(float)
df_Church["c_per"] = df_Church["c_per"].astype(float)
df_Church["d_Hauptberuf"] = df_Church["d_Hauptberuf"].astype(float)

# Create and fill the 'medium' and 'unskilled' columns
df_Church["high"] = ""
# a and b are medium skilled (HISCO: 14130, 14140)
df_Church["medium"] = df_Church["d_Hauptberuf"] * (df_Church["a_per"] + df_Church["b_per"])
# c is low skilled (HISCO: 55140)
df_Church["low"] = df_Church["d_Hauptberuf"] * df_Church["c_per"]
df_Church["unskilled"] = ""

# Reorder the columns
column_order = ["id_inequality", "i_Grossbuchstabe", "i_Ziffer", "d_Hauptberuf_m", "d_Hauptberuf_w", "rb_id", "d_Hauptberuf", "high", "medium", "low", "unskilled"]
df_Church = df_Church[column_order]
df_Church

## Education

In [ ]:
df_Education = occupation_detailed_df[occupation_detailed_df["i_Grossbuchstabe"].isin(["E"])]
df_Education = df_Education[df_Education["i_Ziffer"].isin([4])]

In [ ]:
# Load data
Education_rb = pd.read_excel(f"{classification_path}\\Teachers_by_Provinces.xlsx")

# Calculate the percentages

# For a1
Education_rb["a1"] = Education_rb["a1"].astype(float)
Education_rb["Summe"] = Education_rb["Summe"].astype(float)
Education_rb["a1_per"] = Education_rb["a1"] / Education_rb["Summe"]

# For a2
Education_rb["a2"] = Education_rb["a2"].astype(float)
Education_rb["Summe"] = Education_rb["Summe"].astype(float)
Education_rb["a2_per"] = Education_rb["a2"] / Education_rb["Summe"]

# For b
Education_rb["b"] = Education_rb["b"].astype(float)
Education_rb["Summe"] = Education_rb["Summe"].astype(float)
Education_rb["b_per"] = Education_rb["b"] / Education_rb["Summe"]

# For c
Education_rb["c"] = Education_rb["c"].astype(float)
Education_rb["Summe"] = Education_rb["Summe"].astype(float)
Education_rb["c_per"] = Education_rb["c"] / Education_rb["Summe"]

# Convert 'rb_id' in both DataFrames to string to ensure they have the same format
df_Education["rb_id"] = df_Education["rb_id"].astype(str)
Education_rb["id"] = Education_rb["id"].astype(str)

# Remove the '.0' suffix from the 'rb_id' values in df_A_1
df_Education["rb_id"] = df_Education["rb_id"].str.replace(".0", "", regex=False)

# Perform the merge
df_Education = pd.merge(df_Education, Education_rb[["id", "a1_per", "a2_per", "b_per", "c_per"]], left_on="rb_id", right_on="id", how="left")

# Convert 'a_per', 'b_per', 'c_per', and 'd_Hauptberuf' to float for calculations
df_Education["a1_per"] = df_Education["a1_per"].astype(float)
df_Education["a2_per"] = df_Education["a2_per"].astype(float)
df_Education["b_per"] = df_Education["b_per"].astype(float)
df_Education["c_per"] = df_Education["c_per"].astype(float)
df_Education["d_Hauptberuf"] = df_Education["d_Hauptberuf"].astype(float)

# Create and fill the 'medium' and 'unskilled' columns

# highly skilled teachers (HISCO: )
df_Education["high"] = df_Education["d_Hauptberuf"] * df_Education["a1_per"]
# medium skilled teachers (HISCO: 63220)
df_Education["medium"] = df_Education["d_Hauptberuf"] * df_Education["a2_per"]
# b and c are medium skilled
df_Education["low"] = df_Education["d_Hauptberuf"] * (df_Education["b_per"] + df_Education["c_per"])
df_Education["unskilled"] = ""

# Reorder the columns
column_order = ["id_inequality", "i_Grossbuchstabe", "i_Ziffer", "d_Hauptberuf_m", "d_Hauptberuf_w", "rb_id", "d_Hauptberuf", "high", "medium", "low", "unskilled"]
df_Education = df_Education[column_order]
df_Education

## Health

In [ ]:
df_Health = occupation_detailed_df[occupation_detailed_df["i_Grossbuchstabe"].isin(["E"])]
df_Health = df_Health[df_Health["i_Ziffer"].isin([5])]

In [ ]:
# Load data
Health_rb = pd.read_excel(f"{classification_path}\\Health.xlsx")
# Calculate the percentages

# For a1
Health_rb["a1"] = Health_rb["a1"].astype(float)
Health_rb["Summe"] = Health_rb["Summe"].astype(float)
Health_rb["a1_per"] = Health_rb["a1"] / Health_rb["Summe"]

# For a2
Health_rb["a2"] = Health_rb["a2"].astype(float)
Health_rb["Summe"] = Health_rb["Summe"].astype(float)
Health_rb["a2_per"] = Health_rb["a2"] / Health_rb["Summe"]

# For b
Health_rb["b"] = Health_rb["b"].astype(float)
Health_rb["Summe"] = Health_rb["Summe"].astype(float)
Health_rb["b_per"] = Health_rb["b"] / Health_rb["Summe"]

# For c
Health_rb["c"] = Health_rb["c"].astype(float)
Health_rb["Summe"] = Health_rb["Summe"].astype(float)
Health_rb["c_per"] = Health_rb["c"] / Health_rb["Summe"]

# For d
Health_rb["d"] = Health_rb["d"].astype(float)
Health_rb["Summe"] = Health_rb["Summe"].astype(float)
Health_rb["d_per"] = Health_rb["d"] / Health_rb["Summe"]

# Convert 'rb_id' in both DataFrames to string to ensure they have the same format
df_Health["rb_id"] = df_Health["rb_id"].astype(str)
Health_rb["id"] = Health_rb["id"].astype(str)

# Remove the '.0' suffix from the 'rb_id' values in df_A_1
df_Health["rb_id"] = df_Health["rb_id"].str.replace('.0', '', regex=False)

# Perform the merge 
df_Health = pd.merge(df_Health, Health_rb[["id", "a1_per", "a2_per", "b_per", "c_per", "d_per"]], left_on="rb_id", right_on="id", how="left")

# Convert 'a_per', 'b_per', 'c_per', and 'd_Hauptberuf' to float for calculations
df_Health["a1_per"] = df_Health["a1_per"].astype(float)
df_Health["a2_per"] = df_Health["a2_per"].astype(float)
df_Health["b_per"] = df_Health["b_per"].astype(float)
df_Health["c_per"] = df_Health["c_per"].astype(float)
df_Health["d_per"] = df_Health["d_per"].astype(float)
df_Health["d_Hauptberuf"] = df_Health["d_Hauptberuf"].astype(float)

# Create and fill the 'medium' and 'unskilled' columns

# highly skilled doctors in a1 (HISCO: 06100)
df_Health["high"] = df_Health["d_Hauptberuf"] * df_Health["a1_per"]
# a2 are medium skilled (HISCO: 07310)
df_Health["medium"] = df_Health["d_Hauptberuf"] * df_Health["a2_per"] 
# b, c and d are low (HISCO: 39310, 58240, 59990)
df_Health["low"] = df_Health["d_Hauptberuf"] * (df_Health["b_per"] + df_Health["c_per"] + df_Health["d_per"])
df_Health["unskilled"] = ''

# Reorder the columns
column_order = ["id_inequality", "i_Grossbuchstabe", "i_Ziffer", "d_Hauptberuf_m", "d_Hauptberuf_w", "rb_id", "d_Hauptberuf", "high", "medium", "low", "unskilled"]
df_Health = df_Health[column_order]
df_Health

## Arts

In [ ]:
df_Arts = occupation_detailed_df[occupation_detailed_df["i_Grossbuchstabe"].isin(["E"])]
df_Arts = df_Arts[df_Arts["i_Ziffer"].isin([8])]

In [ ]:
# Load data
Arts_rb = pd.read_excel(f"{classification_path}\\Performing_Arts.xlsx")
# Calculate the percentages

# For a1
Arts_rb["a"] = Arts_rb["a"].astype(float)
Arts_rb["Summe"] = Arts_rb["Summe"].astype(float)
Arts_rb["a_per"] = Arts_rb["a"] / Arts_rb["Summe"]

# For b
Arts_rb["b"] = Arts_rb["b"].astype(float)
Arts_rb["Summe"] = Arts_rb["Summe"].astype(float)
Arts_rb["b_per"] = Arts_rb["b"] / Arts_rb["Summe"]

# For c
Arts_rb["c"] = Arts_rb["c"].astype(float)
Arts_rb["Summe"] = Arts_rb["Summe"].astype(float)
Arts_rb["c_per"] = Arts_rb["c"] / Arts_rb["Summe"]

# Convert 'rb_id' in both DataFrames to string to ensure they have the same format
df_Arts["rb_id"] = df_Arts["rb_id"].astype(str)
Arts_rb["id"] = Arts_rb["id"].astype(str)

# Remove the '.0' suffix from the 'rb_id' values in df_A_1
df_Arts["rb_id"] = df_Arts["rb_id"].str.replace('.0', '', regex=False)

# Perform the merge
df_Arts = pd.merge(df_Arts, Arts_rb[["id", "a_per", "b_per", "c_per"]], left_on="rb_id", right_on="id", how="left")

# Convert 'a_per', 'b_per', 'c_per', and 'd_Hauptberuf' to float for calculations
df_Arts["a_per"] = df_Arts["a_per"].astype(float)
df_Arts["b_per"] = df_Arts["b_per"].astype(float)
df_Arts["c_per"] = df_Arts["c_per"].astype(float)
df_Arts["d_Hauptberuf"] = df_Arts["d_Hauptberuf"].astype(float)

# Create and fill the 'medium' and 'unskilled' columns
df_Arts["high"] = ""
# a and b are medium skilled (HISCO: 63220)
df_Arts["medium"] = df_Arts["d_Hauptberuf"] * (df_Arts["a_per"]+ df_Arts["b_per"])
df_Arts["low"] = df_Arts["d_Hauptberuf"] * df_Arts["c_per"]
df_Arts["unskilled"] = ""

# Reorder the columns
column_order = ["id_inequality", "i_Grossbuchstabe", "i_Ziffer", "d_Hauptberuf_m", "d_Hauptberuf_w", "rb_id", "d_Hauptberuf", "high", "medium", "low", "unskilled"]
df_Arts = df_Arts[column_order]
df_Arts

## Merge them all

In [ ]:
df_Son = pd.concat([df_Forestry, df_Rail, df_Post, df_Army, df_Education, df_Health, df_Arts, df_Church, df_Gov], ignore_index=True)
df_Son

# Merge all data

In [ ]:
def checker(df):
    # Convert columns to float, replacing empty strings with NaN
    for column in ["high", "medium", "low", "unskilled"]:
        df[column] = pd.to_numeric(df[column], errors="coerce")

    # Summing the columns
    a = df["high"].sum()
    b = df["medium"].sum()
    c = df["low"].sum()
    d = df["unskilled"].sum()
    summe = df["d_Hauptberuf"].sum()
    other = a + b + c + d
    diff = summe - other

    print(f"d_Hauptberuf: {summe}, Added skills: {other}, Diff:{diff}")

Marginal differences due to float64

In [ ]:
checker(df_Ind)
checker(df_A_Rest)
checker(df_A_1)
checker(df_Son)
checker(df_Ser)

In [ ]:
df_total = pd.concat([df_Son, df_Ser, df_Ind, df_A_Rest, df_A_1], ignore_index=True)

# Convert columns to numeric types
df_total["high"] = pd.to_numeric(df_total["high"], errors="coerce")
df_total["medium"] = pd.to_numeric(df_total["medium"], errors="coerce")
df_total["low"] = pd.to_numeric(df_total["low"], errors="coerce")
df_total["unskilled"] = pd.to_numeric(df_total["unskilled"], errors="coerce")
df_total["d_Hauptberuf"] = pd.to_numeric(df_total["d_Hauptberuf"], errors="coerce")

# Temporarily replace 0 with NaN in 'd_Hauptberuf' column of df_total
df_total["d_Hauptberuf"].replace(0, np.nan, inplace=True)

# Perform the division to create the new columns in df_total
df_total["per_high"] = df_total["high"] / df_total["d_Hauptberuf"]
df_total["per_medium"] = df_total["medium"] / df_total["d_Hauptberuf"]
df_total["per_low"] = df_total["low"] / df_total["d_Hauptberuf"]
df_total["per_unskilled"] = df_total["unskilled"] / df_total["d_Hauptberuf"]

# Revert NaN back to 0 in 'd_Hauptberuf' of df_total if needed
df_total["d_Hauptberuf"].replace(np.nan, 0, inplace=True)

# Replace NaN in new columns with 0 in df_total
df_total[["per_high", "per_medium", "per_low", "per_unskilled"]] = df_total[["per_high", "per_medium", "per_low", "per_unskilled"]].fillna(0)

df_total

In [ ]:
df_total.to_csv(f"{saving_path}\Total_Dep_Diff_Skills.csv", index=False, sep=",")

In [ ]:
for col in ["d_Hauptberuf", "high", "medium", "low", "unskilled"]:
    df_total[col] = pd.to_numeric(df_total[col], errors="coerce")

grouped_df = df_total.groupby("id_inequality")[["d_Hauptberuf", "high", "medium", "low", "unskilled"]].sum().reset_index()

In [ ]:
filtered_df = df_total[df_total["i_Grossbuchstabe"] == "B"]
grouped_filtered_df = filtered_df.groupby("id_inequality")["d_Hauptberuf"].sum().reset_index()
grouped_filtered_df.rename(columns={"d_Hauptberuf": "Industriearbeiter"}, inplace=True)

In [ ]:
final_df = pd.merge(grouped_df, grouped_filtered_df, on="id_inequality", how="left")

In [ ]:
final_df["Anteil_Industriearbeiter"] = final_df["Industriearbeiter"] / final_df["d_Hauptberuf"]
final_df["per_high"] = final_df["high"] / final_df["d_Hauptberuf"]
final_df["per_medium"] = final_df["medium"] / final_df["d_Hauptberuf"]
final_df["per_low"] = final_df["low"] / final_df["d_Hauptberuf"]
final_df["per_unskilled"] = final_df["unskilled"] / final_df["d_Hauptberuf"]
final_df

In [ ]:
# save final_df on local machine
final_df.to_csv(f"{saving_path}\Main_Dep_Diff_Skills.csv", index=False, sep=",")